In [1]:
import pandas as pd

In [ ]:
assigned = pd.read_csv('assigned_tasks.csv',sep=',')
assigned.info()

unassigned = pd.read_csv('unassigned_tasks.csv',sep=',')
unassigned.info()

techs = pd.read_csv('technicians.csv',sep=',')
techs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Task ID          200 non-null    object 
 1   Technician ID    200 non-null    object 
 2   completion_prob  200 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Task Distance       200 non-null    int64 
 1   Task Priority       200 non-null    int64 
 2   Task Complexity     200 non-null    int64 
 3   Equipment Required  200 non-null    int64 
 4   Customer Rating     200 non-null    int64 
 5   Penalty Cost        200 non-null    int64 
 6   Task ID             200 non-null    object
dtypes: int64(6), object(1)
memory usage: 11.1+ KB
<class 'pandas.core.frame.D

,Technician ID,Eqpt Trained,Tech Complexity
0,T001,0,6
1,T002,0,7
2,T003,1,8
3,T004,1,4
4,T005,0,6
5,T006,0,6
6,T007,1,10
7,T008,0,5
8,T009,1,7
9,T010,1,8


In [23]:
assigned['task_completed'] = (assigned['completion_prob'] > 0.5).astype(int)

merged_df = pd.merge(assigned, unassigned, on='Task ID', how='left', suffixes=('_assigned', '_unassigned'))
# merged_df.info()

full_df = pd.merge(merged_df, techs, on='Technician ID', how='left')
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Task ID             200 non-null    object 
 1   Technician ID       200 non-null    object 
 2   completion_prob     200 non-null    float64
 3   task_completed      200 non-null    int32  
 4   Task Distance       200 non-null    int64  
 5   Task Priority       200 non-null    int64  
 6   Task Complexity     200 non-null    int64  
 7   Equipment Required  200 non-null    int64  
 8   Customer Rating     200 non-null    int64  
 9   Penalty Cost        200 non-null    int64  
 10  Eqpt Trained        200 non-null    int64  
 11  Tech Complexity     200 non-null    int64  
dtypes: float64(1), int32(1), int64(8), object(2)
memory usage: 18.1+ KB


### Task Completion Rate

In [ ]:
avg = full_df['task_completed'].mean()
print(f'Task Completion Rate: {avg:.2%}')

Task Completion Rate: 76.50%


### Expertise Match Rate

In [25]:
def get_expertise_match(row):
    et = row['Eqpt Trained']          # 0 or 1
    er = row['Equipment Required']    # 0 or 1
    tc = row['Task Complexity']       # 1–10
    tech = row['Tech Complexity']     # 1–10

    # task high / tech low → task_comp > tech_comp
    task_high_tech_low = tc > tech
    # task low / tech high → task_comp <= tech_comp
    task_low_tech_high = tc <= tech

    # expertise = 0 cases
    if (
        (et == 1 and er == 1 and task_high_tech_low)
        or (et == 0 and er == 0 and task_high_tech_low)
        or (et == 0 and er == 1 and task_low_tech_high)
        or (et == 1 and er == 0 and task_high_tech_low)
        or (et == 0 and er == 1 and task_high_tech_low)
    ):
        return 0

    # expertise = 1 cases
    elif (
        (et == 1 and er == 1 and task_low_tech_high)
        or (et == 1 and er == 0 and task_low_tech_high)
        or (et == 0 and er == 0 and task_low_tech_high)
    ):
        return 1

    # fallback if no condition matches
    return None

full_df['expertise_match'] = full_df.apply(get_expertise_match, axis=1)

avg_expertise_match = full_df['expertise_match'].mean()
print(f'Average Expertise Match: {avg_expertise_match:.2f}')    


Average Expertise Match: 0.58


### Customer Satistfaction Score

In [ ]:
full_df['new_customer_rating'] = (
    full_df['Customer Rating'] + (full_df['task_completed'] == 1).astype(int)
).clip(upper=5)

avg_rating = full_df['new_customer_rating'].mean()
print(f'Average Customer Satisfaction Score: {avg_rating:.2f}')


Average Customer Satisfaction Score: 3.66


### Total Penalty Cost

In [ ]:
incomplete_tasks = full_df[full_df['task_completed'] == 0]
total_penalty = incomplete_tasks['Penalty Cost'].sum()
print(f'Total Penalty Cost for Incomplete Tasks: ${total_penalty:.2f}')

Total Penalty Cost for Incomplete Tasks: $4139.00
